In [0]:
PROJECT_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Project"
DATASET_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Project/data"
%cd "{PROJECT_PATH}"

/content/drive/My Drive/Deep_Learning_Assignments/Project


In [0]:
# !nvidia-smi
!pip install timm>/tmp/xxy

# Convert to jpg

In [0]:
# !sudo apt install imagemagick
# # /content/drive/My Drive/Deep_Learning_Assignments/Project
# %cd "/content/drive/My Drive/Deep_Learning_Assignments/Project/data/test/"
# !mogrify -format jpg *.jfif
# %cd "/content/drive/My Drive/Deep_Learning_Assignments/Project/data/train/healthy_wheat" 
# !mogrify -format jpg *.jfif  
# %cd "/content/drive/My Drive/Deep_Learning_Assignments/Project/data/train/leaf_rust"
# !mogrify -format jpg *.jfif  
# %cd "/content/drive/My Drive/Deep_Learning_Assignments/Project/data/train/stem_rust" 
# !mogrify -format jpg *.jfif 

# Fastai

In [0]:
from fastai import * 
from fastai.vision import *

In [0]:
# https://docs.fast.ai/vision.data.html#ImageDataBunch.from_folder
data = ImageDataBunch.from_folder(DATASET_PATH, train="train/", test='test/', valid_pct=0.15,
        ds_tfms=get_transforms(flip_vert=True), size=224, num_workers=4, seed=42, bs=32).normalize(imagenet_stats)
# print(data.classes)

In [0]:
# print(data)

In [0]:
# data.show_batch(rows=3, figsize=(15, 15)) 

In [0]:
# https://docs.fast.ai/vision.models.html
# resnet18, resnet34, resnet50, resnet101, resnet152
# squeezenet1_0, squeezenet1_1
# densenet121, densenet169, densenet201, densenet161
# vgg16_bn, vgg19_bn
# alexnet
# %cd over9000/
# try efficientnet later https://www.kaggle.com/khursani8/fast-ai-ootb-cutout-efficientnet
# from over9000 import *
# model.unfreeze()

from timm import create_model

model = create_model('tf_efficientnet_b7', pretrained=True)
in_features = model.classifier.in_features
model.classifier = nn.Linear(in_features=in_features, out_features=3, bias=True)
if torch.cuda.is_available():
  device=torch.device('cuda:0')
else:
  device=torch.device('cpu')
model = model.to(device)

learn = Learner(data=data, model=model, loss_func = nn.CrossEntropyLoss())

In [0]:
# model
saved_model_name = 'fastai-efficientnet-b7-adam-TTA-oversampling'

In [0]:
# model.load('fastai-densenet')
# learn.summary()

In [0]:
# https://github.com/fastai/course-v3/blob/master/nbs/dl1/lesson1-pets.ipynb
# learn.unfreeze()
# print(callbacks)
from fastai.callbacks import *


callbacks_all = [SaveModelCallback(learn, every='improvement', monitor='valid_loss', name = saved_model_name),
             EarlyStoppingCallback(learn, monitor='valid_loss', min_delta=0.001, patience=7),
             OverSamplingCallback(learn)]
             
learn.fit_one_cycle(20, 1e-4, callbacks=callbacks_all)

epoch,train_loss,valid_loss,time
0,1.085227,1.087282,02:48
1,1.070825,1.061512,02:34
2,1.048006,1.026422,02:39
3,1.007282,0.950478,02:42
4,0.919436,0.806095,02:32
5,0.811518,0.606587,02:39
6,0.657489,0.451068,02:34
7,0.503076,0.363340,02:33
8,0.385398,0.314645,02:31
9,0.303976,0.296191,02:42


Better model found at epoch 0 with valid_loss value: 1.0872819423675537.
Better model found at epoch 1 with valid_loss value: 1.0615124702453613.
Better model found at epoch 2 with valid_loss value: 1.0264217853546143.
Better model found at epoch 3 with valid_loss value: 0.9504775404930115.
Better model found at epoch 4 with valid_loss value: 0.8060950636863708.
Better model found at epoch 5 with valid_loss value: 0.6065871715545654.
Better model found at epoch 6 with valid_loss value: 0.451068252325058.
Better model found at epoch 7 with valid_loss value: 0.3633396029472351.
Better model found at epoch 8 with valid_loss value: 0.3146451413631439.
Better model found at epoch 9 with valid_loss value: 0.29619118571281433.
Better model found at epoch 10 with valid_loss value: 0.29368576407432556.
Better model found at epoch 11 with valid_loss value: 0.2935433089733124.
Better model found at epoch 13 with valid_loss value: 0.29281577467918396.
Better model found at epoch 18 with valid_loss

In [0]:
# learn = Learner(data=data, model=model, loss_func = nn.CrossEntropyLoss())
learn.load(saved_model_name)

In [0]:
# preds_test, y_test = model.get_preds(DatasetType.Test)
preds_test,y = learn.TTA(ds_type=DatasetType.Test)

In [0]:
# preds_test, y = preds_test

In [0]:
preds_test.shape

torch.Size([610, 3])

In [0]:
import os
test_path = DATASET_PATH+"/test/test"
# test_path = PROJECT_PATH + "/other-data/test/"
# print(model.data.classes)
submission = pd.DataFrame({'ID': [s.split('.')[0] for s in os.listdir(test_path)  if s.split('.')[1] != 'jfif']})

for i, c in enumerate(learn.data.classes):
  # print(preds_test[:,i].shape)
  # print(c)
  submission[c] = preds_test[:,i]
submission = submission[['ID', 'leaf_rust', 'stem_rust', 'healthy_wheat']] # Get same order as sample sub
submission.head()

,ID,leaf_rust,stem_rust,healthy_wheat
0,HFV4BK,0.998643,0.001058,0.000299
1,0L5TC8,0.976152,0.021554,0.002294
2,VW34Z3,0.358052,0.640065,0.001884
3,HGSMHR,0.011648,0.986721,0.001632
4,0K9T9T,0.114747,0.842226,0.043027


In [0]:
submission.to_csv(PROJECT_PATH+'/submissions/'+saved_model_name+'.csv', index = False)

In [0]:
# !ls -l "{test_path}" | wc -l
# test_path = PROJECT_PATH + "/other-data/test/"
# len(os.listdir(test_path))
print(submission.shape)

(610, 4)


In [0]:
print(learn.data.classes)

['healthy_wheat', 'leaf_rust', 'stem_rust']
